In [2]:
# !pip install pysheds netCDF4 fiona geopandas xarray pyshp

In [3]:
# IMPORT PACKAGES
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap, BoundaryNorm
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
import shapefile
import os

import netCDF4 as nc
from netCDF4 import Dataset
from shapely.geometry import Point, shape, box, mapping
from shapely.vectorized import contains
from shapely.strtree import STRtree

from rasterio.coords import BoundingBox
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.features import shapes

In [4]:
import os
import numpy as np
import geopandas as gpd
import netCDF4 as nc
import matplotlib.pyplot as plt
from shapely.geometry import Point
import shapely

def contains(geom, lon, lat):
    """Check if lon/lat points are within the geometry."""
    points = gpd.GeoSeries([Point(xy) for xy in zip(lon, lat)], crs="EPSG:4326")
    return points.within(geom)

def average_precip_single(shp_input, nc_dir, date_str, plot=True):
    filename = f"IMERG-Final.CLIM.2001-2022.{date_str}.V07B.nc4"
    filepath = os.path.join(nc_dir, filename)

    if not os.path.exists(filepath):
        print("File not found:", filepath)
        return np.nan

    try:
        # Load shapefile and reproject
        shp = gpd.read_file(shp_input).to_crs('EPSG:4326')
        geom = shp.geometry.union_all()  # Updated per deprecation warning

        # Open NetCDF
        with nc.Dataset(filepath) as dataset:
            precip = dataset.variables['precipitation'][:]
            lat = dataset.variables['lat'][:]
            lon = dataset.variables['lon'][:]

        # Handle time dimension
        if precip.ndim == 3:
            precip = precip.mean(axis=0)

        # Get bounding box of shape
        minlon, minlat, maxlon, maxlat = geom.bounds

        # Create lat/lon masks
        lat_mask = (lat >= minlat) & (lat <= maxlat)
        lon_mask = (lon >= minlon) & (lon <= maxlon)

        # Safe mask check
        if not lat_mask.any() or not lon_mask.any():
            print("No lat/lon points within shapefile bounds.")
            return np.nan

        # Slice arrays safely
        lat_filtered = lat[lat_mask]
        lon_filtered = lon[lon_mask]

        if precip.shape[0] < lat_mask.sum() or precip.shape[1] < lon_mask.sum():
            print("Filtered indices exceed precip array dimensions.")
            return np.nan

        precip_filtered = precip[np.ix_(lat_mask, lon_mask)]

        # Create grids
        lon_grid, lat_grid = np.meshgrid(lon_filtered, lat_filtered)

        # Optional: visualize the contour plot
        if plot:
            fig, ax = plt.subplots(figsize=(8, 6))
            ctf = ax.contourf(lon_grid, lat_grid, precip_filtered, cmap='viridis', levels=15)
            shp.boundary.plot(ax=ax, color='red', linewidth=1)
            plt.colorbar(ctf, ax=ax, label='Precipitation (mm)')
            ax.set_title(f'Precipitation - {date_str}')
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            plt.tight_layout()
            plt.show()

        # Flatten and apply geometry mask
        lon_flat = lon_grid.ravel()
        lat_flat = lat_grid.ravel()
        precip_flat = precip_filtered.ravel()

        mask = contains(geom, lon_flat, lat_flat)

        if not np.any(mask):
            print("No points inside the geometry.")
            return np.nan

        avg_precip = np.nanmean(precip_flat[mask])
        return avg_precip

    except Exception as e:
        print(f"Error for {date_str}, {shp_input}: {e}")
        return np.nan

In [5]:
opened = []

file_path = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/ALL_STATIONS_ALL_MONTHS.csv'

# Export to a CSV file
stations_df = pd.read_csv(file_path)
# print(stations_df)

In [6]:
precip = np.zeros(len(stations_df))
    
for i in range(0,1):
# i = 0
    
    date = stations_df['Date']
    number = stations_df['GRDC_No'][i]
    shp_log = stations_df['SHP']
    
    # Read the shapefiles
    shapefile1 = f'/global/home/users/arvalcarcel/ondemand/data/dem/{number}/{number}.shp' # delineated shapefile # delineated shapefile
    shapefile2 = f'/global/scratch/users/arvalcarcel/CSMUB/DATA/SHAPEFILES/{number}/{number}.shp' # GRDC shapefile
    
    if shp_log[i] == 1:
        shapefile = shapefile1
    
    elif shp_log[i] == 2:
        shapefile = shapefile2
    
    # print(shapefile)
    
    precip_folder = '/global/scratch/users/arvalcarcel/CSMUB/DATA/PRECIP/RESAMPLED/'
    date_input = date[i][-2:]
    # print(date_input)


    # ==== INPUT FILES ====
    shapefile_path = shapefile
    netcdf_path = precip_folder + 'IMERG-Final.CLIM.2001-2022.08.V07B.nc4'  # Update to your date/month

    # ==== LOAD SHAPEFILE ====
    shp = gpd.read_file(shapefile_path).to_crs("EPSG:4326")
    geom = shp.geometry.union_all()
    
    # ==== LOAD NETCDF DATA ====
    ds = nc.Dataset(netcdf_path)
    precip = ds.variables['precipitation'][:]
    lat = ds.variables['lat'][:]
    lon = ds.variables['lon'][:]
    
    # ==== HANDLE TIME DIMENSION ====
    if precip.ndim == 3:
        precip = precip.mean(axis=0)
    
    # ==== BOUNDING BOX MASK ====
   # ==== Check and sort longitudes if needed ====
    if not np.all(np.diff(lon) > 0):
        print("Sorting longitudes...")
        sort_idx = np.argsort(lon)
        lon = lon[sort_idx]
        precip = precip[:, sort_idx]  # shape: (lat, lon)

    # ==== Apply masking ====
    minlon, minlat, maxlon, maxlat = geom.bounds
    lat_mask = (lat >= minlat) & (lat <= maxlat)
    lon_mask = (lon >= minlon) & (lon <= maxlon)
    
    lat_idx = np.where(lat_mask)[0]
    lon_idx = np.where(lon_mask)[0]
    
    if len(lat_idx) == 0 or len(lon_idx) == 0:
        print("No valid lat/lon cells within shape bounds.")
        exit()
    
    # Validate indexing
    if lat_idx.max() >= precip.shape[0] or lon_idx.max() >= precip.shape[1]:
        print("Lat/lon indices exceed precipitation array shape.")
        print("Lat index max:", lat_idx.max(), "Lat shape:", precip.shape[0])
        print("Lon index max:", lon_idx.max(), "Lon shape:", precip.shape[1])
        exit()
    
    # Final extraction
    lat_filtered = lat[lat_idx]
    lon_filtered = lon[lon_idx]
    precip_filtered = precip[np.ix_(lat_idx, lon_idx)]


    
    # ==== CREATE MESHGRID ====
    lon_grid, lat_grid = np.meshgrid(lon_filtered, lat_filtered)
    
    # ==== PLOTTING ====
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Use pcolormesh or contourf (contourf is smoother)
    cf = ax.contourf(lon_grid, lat_grid, precip_filtered, cmap='viridis', levels=20)
    shp.boundary.plot(ax=ax, color='red', linewidth=1)
    
    # Add labels
    plt.colorbar(cf, ax=ax, label='Precipitation (mm)')
    ax.set_title('Precipitation over Shapefile Area')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # # Compute avg precip for this date and shapefile
    # avg = average_precip_single(shapefile, precip_folder, date_input, plot=True)
    # # print(avg)
    # # Assign result back to the matching rows
    # precip[i] = avg*1e6
    # # print(precip[i])

Lat/lon indices exceed precipitation array shape.
Lat index max: 682 Lat shape: 3599
Lon index max: 2102 Lon shape: 1799


IndexError: index 1965 is out of bounds for axis 1 with size 1799

In [6]:
stations_df['P'] = precip

In [7]:
print(stations_df)

       Unnamed: 0     Date       Q       SWE    SWE_scaled  GRDC_No  SHP  \
0               0  2018-08  24.526  2.087791  1.809042e+06  1159100    1   
1               1  2018-09  31.372  1.835435  1.590379e+06  1159100    1   
2               2  2018-10  19.572  1.976332  1.712464e+06  1159100    1   
3               3  2018-11   7.349  1.633273  1.415208e+06  1159100    1   
4               4  2018-12  13.824  1.782850  1.544815e+06  1159100    1   
...           ...      ...     ...       ...           ...      ...  ...   
23053          57  2023-05   3.046  3.400061  5.610100e+04  6594090    1   
23054          58  2023-06   2.489  3.195970  5.273351e+04  6594090    1   
23055          59  2023-07   2.021  3.656259  6.032828e+04  6594090    2   
23056          60  2023-08   2.067  3.578097  5.903860e+04  6594090    1   
23057          61  2023-09   2.483  3.630205  5.989839e+04  6594090    1   

           Area  Latitude  Avg Slope  Max Slope     Aridity    Precip    P  
0      866

In [8]:
# opened = []

# csv_path = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/CSV/'
# masterlist = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/ALL_STATIONS_REVISED_PRECIP.csv'

# # Load the master list of stations
# stations_df = pd.read_csv(masterlist)

# # Extract station numbers, areas, and latitudes
# station_num = stations_df['grdc_no']
# station_shp = stations_df['shapefile_code']
# station_area = stations_df['area']
# station_lat = stations_df['lat']
# station_avgslope = stations_df['avg_slope']
# station_maxslope = stations_df['max_slope']
# station_aridity = stations_df['avg_aridity']
# station_precip = stations_df['P']

# # Map station numbers to areas and latitudes
# station_area_map = dict(zip(station_num, station_area))
# station_lat_map = dict(zip(station_num, station_lat))
# station_avgslope_map = dict(zip(station_num, station_avgslope))
# station_maxslope_map = dict(zip(station_num, station_maxslope))
# station_aridity_map = dict(zip(station_num, station_aridity))
# # station_precip_map = dict(zip(station_num, station_precip))

# # Generate the list of file paths
# arrayFile = [os.path.join(csv_path, f"{station_no}.csv") for station_no in station_num]
# # print(arrayFile)
# # Initialize a list to store opened DataFrames
# for file in arrayFile:
#     station_no = os.path.basename(file).split('.')[0]
#     # print(station_no)# Extract station number from the filename
#     if os.path.exists(file):  # Check if file exists
#         df = pd.read_csv(file, index_col=None, header=0)
#         station_no_int = int(station_no)  # Convert station number to integer for lookup
#         df['GRDC_No'] = station_no_int  # Add the station number as a new column
#         df['SHP'] = station_shp
#         df['Area'] = station_area_map.get(station_no_int, None)  # Add the Area column
#         df['Latitude'] = station_lat_map.get(station_no_int, None)  # Add the latitude column
#         df['Avg Slope'] = station_avgslope_map.get(station_no_int, None)
#         df['Max Slope'] = station_maxslope_map.get(station_no_int, None)
#         df['Aridity'] = station_aridity_map.get(station_no_int, None)
#         df['Precip'] = station_precip
#         opened.append(df)

# # Combine all DataFrames into one
# total_df = pd.concat(opened, axis=0, ignore_index=True)

# # Print or save the resulting DataFrame
# print(total_df)
# total_df.to_csv(file_path, index=False)